In [5]:
from __future__ import print_function
import numpy as np
import cntk as C
from cntk.learners import sgd, learning_rate_schedule, UnitType
from cntk.logging import ProgressPrinter
from cntk.layers import Dense, Sequential

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def generate_random_data(sample_size, feature_dim, num_classes):
     # Create synthetic data using NumPy.
     Y = np.random.randint(size=(sample_size, 1), low=0, high=num_classes)

     # Make sure that the data is separable
     X = (np.random.randn(sample_size, feature_dim) + 3) * (Y + 1)
     X = X.astype(np.float32)
     # converting class 0 into the vector "1 0 0",
     # class 1 into vector "0 1 0", ...
     class_ind = [Y == class_number for class_number in range(num_classes)]
     Y = np.asarray(np.hstack(class_ind), dtype=np.float32)
     return X, Y

def ffnet(args):
    lr, hd = args
    inputs = 2
    outputs = 2
    layers = 2
    hidden_dimension = hd

    # input variables denoting the features and label data
    features = C.input((inputs), np.float32)
    label = C.input((outputs), np.float32)

    # Instantiate the feedforward classification model
    my_model = Sequential ([
                    Dense(hidden_dimension, activation=C.sigmoid),
                    Dense(outputs)])
    z = my_model(features)

    ce = C.cross_entropy_with_softmax(z, label)
    pe = C.classification_error(z, label)

    # Instantiate the trainer object to drive the model training
    lr_per_minibatch = learning_rate_schedule(lr, UnitType.minibatch)
    progress_printer = ProgressPrinter(0)
    trainer = C.Trainer(z, (ce, pe), [sgd(z.parameters, lr=lr_per_minibatch)], [progress_printer])

    # Get minibatches of training data and perform model training
    minibatch_size = 25
    num_minibatches_to_train = 1024
    aggregate_loss = 0.0
    print ("Learning rate ", lr, "; Hidden dimension ", hd)
    for i in range(num_minibatches_to_train):
        train_features, labels = generate_random_data(minibatch_size, inputs, outputs)
        # Specify the mapping of input variables in the model to actual minibatch data to be trained with
        trainer.train_minibatch({features : train_features, label : labels})
        sample_count = trainer.previous_minibatch_sample_count
        aggregate_loss += trainer.previous_minibatch_loss_average * sample_count

    last_avg_error = aggregate_loss / trainer.total_number_of_samples_seen

    test_features, test_labels = generate_random_data(minibatch_size, inputs, outputs)
    avg_error = trainer.test_minibatch({features : test_features, label : test_labels})
    print(' error rate on an unseen minibatch: {}'.format(avg_error))
    return avg_error

np.random.seed(98052)
trials = Trials()
best = fmin(ffnet,
    space=[hp.uniform('lr', 0.1, 0.2), hp.quniform('hd', 20, 100, 10)],
    algo=tpe.suggest,
    max_evals=10,
    trials=trials)

print (best)

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate  0.1901723549474807 ; Hidden dimension  30.0
Learning rate per minibatch: 0.1901723549474807
      1.4        1.4       0.56       0.56            25
    0.844      0.567      0.307       0.18            75
    0.705        0.6      0.297       0.29           175
    0.609      0.526      0.264      0.235           375
    0.543       0.48      0.206      0.152           775
    0.473      0.406      0.163       0.12          1575
    0.397      0.322      0.132      0.101          3175
    0.333      0.268      0.113     0.0953          6375
    0.282      0.231     0.0965     0.0797         12775
    0.252      0.223     0.0898     0.0831         25575
 error rate on an unseen minibatch: 0.0
 average      since    average      since      examples
    loss       last     metric       last              
 --

    0.492      0.399      0.215      0.138          1575
    0.407      0.324       0.16      0.106          3175
    0.335      0.263      0.126     0.0922          6375
    0.285      0.236      0.107     0.0886         12775
    0.251      0.217      0.094     0.0808         25575
 error rate on an unseen minibatch: 0.0
{'hd': 30.0, 'lr': 0.1901723549474807}
